In [ ]:
"""
Hierarchical Clustering on Sales Data Sample
Dataset: sales_data_sample.csv from Kaggle
Includes: Elbow Method, Dendrogram, Cluster Analysis
"""

# Install required packages
!pip install pandas numpy matplotlib seaborn scikit-learn scipy -q

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import pdist, squareform
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import warnings
warnings.filterwarnings('ignore')

print("=" * 100)
print("HIERARCHICAL CLUSTERING ON SALES DATA - WITH ELBOW METHOD")
print("=" * 100)

# ============================================================================
# 1. LOAD AND EXPLORE DATASET
# ============================================================================
print("\n[1/7] Loading Dataset...")

# Upload the dataset
from google.colab import files
print("\n📁 Please upload your 'sales_data_sample.csv' file when prompted...")
uploaded = files.upload()

# Load the dataset
df = pd.read_csv('sales_data_sample.csv', encoding='latin-1')

print(f"\n✓ Dataset loaded successfully!")
print(f"Dataset shape: {df.shape}")
print(f"\nColumn names:")
print(df.columns.tolist())

print(f"\nFirst few rows:")
print(df.head())

print(f"\nDataset Information:")
print(df.info())

print(f"\nStatistical Summary:")
print(df.describe())

print(f"\nMissing Values:")
missing = df.isnull().sum()
print(missing[missing > 0])

# ============================================================================
# 2. DATA PREPROCESSING
# ============================================================================
print("\n" + "="*100)
print("[2/7] Data Preprocessing...")
print("="*100)

# Select relevant numerical features for clustering
numerical_features = ['QUANTITYORDERED', 'PRICEEACH', 'SALES', 'MSRP']

# Check if columns exist, adjust if needed
available_numerical = [col for col in numerical_features if col in df.columns]
print(f"\nAvailable numerical features: {available_numerical}")

# Create a working dataframe with numerical features
df_cluster = df[available_numerical].copy()

# Handle missing values
df_cluster = df_cluster.dropna()

print(f"\nDataset shape after cleaning: {df_cluster.shape}")
print(f"\nFeatures used for clustering:")
for feature in df_cluster.columns:
    print(f"  - {feature}: mean={df_cluster[feature].mean():.2f}, std={df_cluster[feature].std():.2f}")

# Add additional features if available
if 'ORDERDATE' in df.columns:
    # Extract year and month from order date
    try:
        df['ORDERDATE'] = pd.to_datetime(df['ORDERDATE'])
        df_cluster['YEAR'] = df['ORDERDATE'].dt.year
        df_cluster['MONTH'] = df['ORDERDATE'].dt.month
        print(f"\n✓ Added temporal features: YEAR, MONTH")
    except:
        pass

# Handle categorical variables if needed for enriched clustering
categorical_features = []
if 'PRODUCTLINE' in df.columns:
    le = LabelEncoder()
    df_cluster['PRODUCTLINE_ENCODED'] = le.fit_transform(df['PRODUCTLINE'].fillna('Unknown'))
    categorical_features.append('PRODUCTLINE')
    print(f"\n✓ Encoded categorical feature: PRODUCTLINE")

if 'COUNTRY' in df.columns:
    le_country = LabelEncoder()
    df_cluster['COUNTRY_ENCODED'] = le_country.fit_transform(df['COUNTRY'].fillna('Unknown'))
    categorical_features.append('COUNTRY')
    print(f"\n✓ Encoded categorical feature: COUNTRY")

print(f"\nFinal feature set shape: {df_cluster.shape}")
print(f"Features: {df_cluster.columns.tolist()}")

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_cluster)

print(f"\n✓ Features standardized (mean=0, std=1)")

# ============================================================================
# 3. ELBOW METHOD - DETERMINE OPTIMAL CLUSTERS
# ============================================================================
print("\n" + "="*100)
print("[3/7] Applying Elbow Method...")
print("="*100)

# We'll use different methods to determine optimal clusters
max_clusters = 15

# Method 1: Within-Cluster Sum of Squares (WCSS) - Similar to K-means elbow
from sklearn.cluster import AgglomerativeClustering

wcss = []
silhouette_scores = []
calinski_scores = []
davies_bouldin_scores = []

print("\nCalculating metrics for different numbers of clusters...")
print("Clusters | WCSS      | Silhouette | Calinski-Harabasz | Davies-Bouldin")
print("-" * 75)

for n_clusters in range(2, max_clusters + 1):
    # Fit Agglomerative Clustering
    clusterer = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward')
    cluster_labels = clusterer.fit_predict(X_scaled)

    # Calculate WCSS
    cluster_centers = []
    for i in range(n_clusters):
        cluster_points = X_scaled[cluster_labels == i]
        if len(cluster_points) > 0:
            cluster_centers.append(cluster_points.mean(axis=0))

    wcss_value = 0
    for i in range(n_clusters):
        cluster_points = X_scaled[cluster_labels == i]
        if len(cluster_points) > 0 and i < len(cluster_centers):
            wcss_value += np.sum((cluster_points - cluster_centers[i])**2)

    wcss.append(wcss_value)

    # Calculate Silhouette Score
    sil_score = silhouette_score(X_scaled, cluster_labels)
    silhouette_scores.append(sil_score)

    # Calculate Calinski-Harabasz Score
    cal_score = calinski_harabasz_score(X_scaled, cluster_labels)
    calinski_scores.append(cal_score)

    # Calculate Davies-Bouldin Score
    db_score = davies_bouldin_score(X_scaled, cluster_labels)
    davies_bouldin_scores.append(db_score)

    print(f"{n_clusters:8d} | {wcss_value:9.2f} | {sil_score:10.4f} | {cal_score:17.2f} | {db_score:14.4f}")

# ============================================================================
# 4. HIERARCHICAL CLUSTERING WITH OPTIMAL CLUSTERS
# ============================================================================
print("\n" + "="*100)
print("[4/7] Performing Hierarchical Clustering...")
print("="*100)

# Create linkage matrix for dendrogram
print("\nComputing linkage matrix using Ward's method...")
linkage_matrix = linkage(X_scaled, method='ward')
print("✓ Linkage matrix computed!")

# Determine optimal number of clusters from elbow method
# Calculate the rate of change in WCSS
wcss_diff = np.diff(wcss)
wcss_diff_ratio = wcss_diff[:-1] / wcss_diff[1:]

# Find the elbow point
optimal_clusters_wcss = np.argmax(wcss_diff_ratio) + 3  # +3 because we start from 2 and use diff

# Also use silhouette score
optimal_clusters_silhouette = np.argmax(silhouette_scores) + 2

print(f"\n📊 Optimal Clusters Analysis:")
print(f"   Based on WCSS Elbow Method: {optimal_clusters_wcss} clusters")
print(f"   Based on Silhouette Score: {optimal_clusters_silhouette} clusters")
print(f"   Based on Calinski-Harabasz: {np.argmax(calinski_scores) + 2} clusters")
print(f"   Based on Davies-Bouldin: {np.argmin(davies_bouldin_scores) + 2} clusters")

# Use the most common suggestion or silhouette score
optimal_k = optimal_clusters_silhouette
print(f"\n✓ Selected optimal number of clusters: {optimal_k}")

# Form clusters
clusters = fcluster(linkage_matrix, optimal_k, criterion='maxclust')

print(f"\n📈 Cluster Distribution:")
unique, counts = np.unique(clusters, return_counts=True)
for cluster_id, count in zip(unique, counts):
    percentage = (count / len(clusters)) * 100
    print(f"   Cluster {cluster_id}: {count} samples ({percentage:.2f}%)")

# Add clusters to dataframe
df_cluster['Cluster'] = clusters
df['Cluster'] = clusters

# ============================================================================
# 5. CLUSTER ANALYSIS
# ============================================================================
print("\n" + "="*100)
print("[5/7] Analyzing Clusters...")
print("="*100)

# Analyze each cluster
print("\n📊 CLUSTER CHARACTERISTICS:")
print("="*100)

for cluster_id in sorted(df_cluster['Cluster'].unique()):
    cluster_data = df_cluster[df_cluster['Cluster'] == cluster_id]

    print(f"\n{'='*80}")
    print(f"CLUSTER {cluster_id} - {len(cluster_data)} samples ({len(cluster_data)/len(df_cluster)*100:.2f}%)")
    print(f"{'='*80}")

    # Statistical summary
    for feature in available_numerical:
        mean_val = cluster_data[feature].mean()
        median_val = cluster_data[feature].median()
        std_val = cluster_data[feature].std()
        print(f"{feature:20s}: Mean={mean_val:10.2f}, Median={median_val:10.2f}, Std={std_val:10.2f}")

    # Additional insights if categorical data available
    if 'PRODUCTLINE' in df.columns:
        print(f"\nTop Product Lines:")
        top_products = df[df['Cluster'] == cluster_id]['PRODUCTLINE'].value_counts().head(3)
        for product, count in top_products.items():
            print(f"   {product}: {count} ({count/len(cluster_data)*100:.1f}%)")

    if 'COUNTRY' in df.columns:
        print(f"\nTop Countries:")
        top_countries = df[df['Cluster'] == cluster_id]['COUNTRY'].value_counts().head(3)
        for country, count in top_countries.items():
            print(f"   {country}: {count} ({count/len(cluster_data)*100:.1f}%)")

# ============================================================================
# 6. VISUALIZATIONS
# ============================================================================
print("\n" + "="*100)
print("[6/7] Creating Visualizations...")
print("="*100)

# Create comprehensive visualization
fig = plt.figure(figsize=(24, 18))

# Plot 1: Elbow Method - WCSS
ax1 = plt.subplot(3, 4, 1)
ax1.plot(range(2, max_clusters + 1), wcss, 'bo-', linewidth=2, markersize=8)
ax1.axvline(x=optimal_clusters_wcss, color='r', linestyle='--', linewidth=2,
            label=f'Optimal k={optimal_clusters_wcss}')
ax1.set_xlabel('Number of Clusters', fontsize=11)
ax1.set_ylabel('Within-Cluster Sum of Squares (WCSS)', fontsize=11)
ax1.set_title('Elbow Method - WCSS', fontsize=13, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Silhouette Score
ax2 = plt.subplot(3, 4, 2)
ax2.plot(range(2, max_clusters + 1), silhouette_scores, 'go-', linewidth=2, markersize=8)
ax2.axvline(x=optimal_clusters_silhouette, color='r', linestyle='--', linewidth=2,
            label=f'Optimal k={optimal_clusters_silhouette}')
ax2.set_xlabel('Number of Clusters', fontsize=11)
ax2.set_ylabel('Silhouette Score', fontsize=11)
ax2.set_title('Silhouette Score (Higher is Better)', fontsize=13, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Calinski-Harabasz Score
ax3 = plt.subplot(3, 4, 3)
ax3.plot(range(2, max_clusters + 1), calinski_scores, 'mo-', linewidth=2, markersize=8)
ax3.set_xlabel('Number of Clusters', fontsize=11)
ax3.set_ylabel('Calinski-Harabasz Score', fontsize=11)
ax3.set_title('Calinski-Harabasz Index (Higher is Better)', fontsize=13, fontweight='bold')
ax3.grid(True, alpha=0.3)

# Plot 4: Davies-Bouldin Score
ax4 = plt.subplot(3, 4, 4)
ax4.plot(range(2, max_clusters + 1), davies_bouldin_scores, 'ro-', linewidth=2, markersize=8)
ax4.set_xlabel('Number of Clusters', fontsize=11)
ax4.set_ylabel('Davies-Bouldin Score', fontsize=11)
ax4.set_title('Davies-Bouldin Index (Lower is Better)', fontsize=13, fontweight='bold')
ax4.grid(True, alpha=0.3)

# Plot 5: Dendrogram (Full)
ax5 = plt.subplot(3, 4, (5, 8))
dendrogram(linkage_matrix, ax=ax5, color_threshold=linkage_matrix[-(optimal_k-1), 2],
           above_threshold_color='gray')
ax5.axhline(y=linkage_matrix[-(optimal_k-1), 2], color='r', linestyle='--', linewidth=2,
            label=f'{optimal_k} clusters')
ax5.set_xlabel('Sample Index', fontsize=11)
ax5.set_ylabel('Distance (Ward)', fontsize=11)
ax5.set_title(f'Hierarchical Clustering Dendrogram (Full)', fontsize=13, fontweight='bold')
ax5.legend()

# Plot 6: Dendrogram (Truncated)
ax6 = plt.subplot(3, 4, (9, 12))
dendrogram(linkage_matrix, ax=ax6, truncate_mode='lastp', p=30,
           color_threshold=linkage_matrix[-(optimal_k-1), 2],
           above_threshold_color='gray')
ax6.axhline(y=linkage_matrix[-(optimal_k-1), 2], color='r', linestyle='--', linewidth=2,
            label=f'{optimal_k} clusters')
ax6.set_xlabel('Cluster Size', fontsize=11)
ax6.set_ylabel('Distance (Ward)', fontsize=11)
ax6.set_title(f'Hierarchical Clustering Dendrogram (Truncated - Last 30 Merges)',
             fontsize=13, fontweight='bold')
ax6.legend()

plt.tight_layout()
plt.savefig('hierarchical_clustering_elbow_dendrogram.png', dpi=300, bbox_inches='tight')
print("✓ Saved: hierarchical_clustering_elbow_dendrogram.png")
plt.show()

# Second figure: Cluster Analysis
fig2 = plt.figure(figsize=(24, 14))

# PCA for visualization
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

print(f"\nPCA Explained Variance: {pca.explained_variance_ratio_[0]:.2%} + {pca.explained_variance_ratio_[1]:.2%} = {sum(pca.explained_variance_ratio_):.2%}")

# Plot 1: PCA Scatter Plot with Clusters
ax1 = plt.subplot(2, 3, 1)
scatter = ax1.scatter(X_pca[:, 0], X_pca[:, 1], c=clusters, cmap='viridis',
                     s=50, alpha=0.6, edgecolors='black', linewidth=0.5)
ax1.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance)', fontsize=11)
ax1.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance)', fontsize=11)
ax1.set_title(f'Clusters Visualization (PCA) - {optimal_k} Clusters', fontsize=13, fontweight='bold')
plt.colorbar(scatter, ax=ax1, label='Cluster')
ax1.grid(True, alpha=0.3)

# Plot 2: Cluster Size Distribution
ax2 = plt.subplot(2, 3, 2)
cluster_counts = df_cluster['Cluster'].value_counts().sort_index()
colors = plt.cm.viridis(np.linspace(0, 1, len(cluster_counts)))
ax2.bar(cluster_counts.index, cluster_counts.values, color=colors, edgecolor='black', alpha=0.7)
ax2.set_xlabel('Cluster', fontsize=11)
ax2.set_ylabel('Number of Samples', fontsize=11)
ax2.set_title('Cluster Size Distribution', fontsize=13, fontweight='bold')
ax2.grid(axis='y', alpha=0.3)
for i, v in enumerate(cluster_counts.values):
    ax2.text(cluster_counts.index[i], v + 10, str(v), ha='center', fontweight='bold')

# Plot 3: Feature Comparison by Cluster (Box Plot)
ax3 = plt.subplot(2, 3, 3)
cluster_data_plot = df_cluster[['SALES', 'Cluster']].copy() if 'SALES' in available_numerical else df_cluster[[available_numerical[0], 'Cluster']].copy()
feature_name = 'SALES' if 'SALES' in available_numerical else available_numerical[0]
sns.boxplot(x='Cluster', y=feature_name, data=cluster_data_plot, palette='viridis', ax=ax3)
ax3.set_xlabel('Cluster', fontsize=11)
ax3.set_ylabel(feature_name, fontsize=11)
ax3.set_title(f'{feature_name} Distribution by Cluster', fontsize=13, fontweight='bold')
ax3.grid(axis='y', alpha=0.3)

# Plot 4: Sales vs Quantity by Cluster
if 'SALES' in available_numerical and 'QUANTITYORDERED' in available_numerical:
    ax4 = plt.subplot(2, 3, 4)
    for cluster_id in sorted(df_cluster['Cluster'].unique()):
        cluster_subset = df_cluster[df_cluster['Cluster'] == cluster_id]
        ax4.scatter(cluster_subset['QUANTITYORDERED'], cluster_subset['SALES'],
                   label=f'Cluster {cluster_id}', alpha=0.6, s=50)
    ax4.set_xlabel('Quantity Ordered', fontsize=11)
    ax4.set_ylabel('Sales', fontsize=11)
    ax4.set_title('Sales vs Quantity by Cluster', fontsize=13, fontweight='bold')
    ax4.legend()
    ax4.grid(True, alpha=0.3)

# Plot 5: Average Feature Values by Cluster (Heatmap)
ax5 = plt.subplot(2, 3, 5)
cluster_means = df_cluster.groupby('Cluster')[available_numerical].mean()
sns.heatmap(cluster_means.T, annot=True, fmt='.2f', cmap='YlOrRd', ax=ax5,
            cbar_kws={'label': 'Average Value'})
ax5.set_xlabel('Cluster', fontsize=11)
ax5.set_ylabel('Features', fontsize=11)
ax5.set_title('Average Feature Values by Cluster', fontsize=13, fontweight='bold')

# Plot 6: Product Line Distribution by Cluster (if available)
if 'PRODUCTLINE' in df.columns:
    ax6 = plt.subplot(2, 3, 6)
    product_cluster = pd.crosstab(df['Cluster'], df['PRODUCTLINE'])
    product_cluster_pct = product_cluster.div(product_cluster.sum(axis=1), axis=0) * 100
    product_cluster_pct.T.plot(kind='bar', stacked=True, ax=ax6, colormap='viridis')
    ax6.set_xlabel('Product Line', fontsize=11)
    ax6.set_ylabel('Percentage', fontsize=11)
    ax6.set_title('Product Line Distribution by Cluster (%)', fontsize=13, fontweight='bold')
    ax6.legend(title='Cluster', bbox_to_anchor=(1.05, 1), loc='upper left')
    ax6.set_xticklabels(ax6.get_xticklabels(), rotation=45, ha='right')
else:
    # Alternative: Price vs MSRP
    if 'PRICEEACH' in available_numerical and 'MSRP' in available_numerical:
        ax6 = plt.subplot(2, 3, 6)
        for cluster_id in sorted(df_cluster['Cluster'].unique()):
            cluster_subset = df_cluster[df_cluster['Cluster'] == cluster_id]
            ax6.scatter(cluster_subset['PRICEEACH'], cluster_subset['MSRP'],
                       label=f'Cluster {cluster_id}', alpha=0.6, s=50)
        ax6.set_xlabel('Price Each', fontsize=11)
        ax6.set_ylabel('MSRP', fontsize=11)
        ax6.set_title('Price vs MSRP by Cluster', fontsize=13, fontweight='bold')
        ax6.legend()
        ax6.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('hierarchical_clustering_analysis.png', dpi=300, bbox_inches='tight')
print("✓ Saved: hierarchical_clustering_analysis.png")
plt.show()

# ============================================================================
# 7. EXPORT RESULTS
# ============================================================================
print("\n" + "="*100)
print("[7/7] Exporting Results...")
print("="*100)

# Save clustered data
df_export = df.copy()
df_export['Cluster'] = clusters
df_export.to_csv('sales_data_clustered.csv', index=False)
print("✓ Clustered data exported to: sales_data_clustered.csv")

# Create summary report
summary_data = []
for cluster_id in sorted(df_cluster['Cluster'].unique()):
    cluster_subset = df_cluster[df_cluster['Cluster'] == cluster_id]
    summary = {
        'Cluster': cluster_id,
        'Count': len(cluster_subset),
        'Percentage': f"{len(cluster_subset)/len(df_cluster)*100:.2f}%"
    }
    for feature in available_numerical:
        summary[f'{feature}_mean'] = cluster_subset[feature].mean()
        summary[f'{feature}_median'] = cluster_subset[feature].median()
    summary_data.append(summary)

summary_df = pd.DataFrame(summary_data)
summary_df.to_excel('cluster_summary.xlsx', index=False)
print("✓ Cluster summary exported to: cluster_summary.xlsx")

# Create metrics report
metrics_df = pd.DataFrame({
    'Number of Clusters': range(2, max_clusters + 1),
    'WCSS': wcss,
    'Silhouette Score': silhouette_scores,
    'Calinski-Harabasz': calinski_scores,
    'Davies-Bouldin': davies_bouldin_scores
})
metrics_df.to_csv('clustering_metrics.csv', index=False)
print("✓ Clustering metrics exported to: clustering_metrics.csv")

# Final Report
print("\n" + "="*100)
print("📄 FINAL CLUSTERING REPORT")
print("="*100)
print(f"\nDataset: Sales Data Sample")
print(f"Total Samples: {len(df)}")
print(f"Features Used: {len(df_cluster.columns) - 1}")
print(f"Feature Names: {', '.join([f for f in df_cluster.columns if f != 'Cluster'])}")
print(f"\nLinkage Method: Ward")
print(f"Optimal Number of Clusters: {optimal_k}")
print(f"\nClustering Quality Metrics:")
print(f"   Silhouette Score: {silhouette_scores[optimal_k-2]:.4f}")
print(f"   Calinski-Harabasz: {calinski_scores[optimal_k-2]:.2f}")
print(f"   Davies-Bouldin: {davies_bouldin_scores[optimal_k-2]:.4f}")

print(f"\n📊 Cluster Distribution:")
for cluster_id in sorted(df_cluster['Cluster'].unique()):
    count = len(df_cluster[df_cluster['Cluster'] == cluster_id])
    pct = count / len(df_cluster) * 100
    print(f"   Cluster {cluster_id}: {count:5d} samples ({pct:5.2f}%)")

print("\n" + "="*100)
print("✅ HIERARCHICAL CLUSTERING ANALYSIS COMPLETE!")
print("="*100)
print("\nGenerated Files:")
print("  1. hierarchical_clustering_elbow_dendrogram.png")
print("  2. hierarchical_clustering_analysis.png")
print("  3. sales_data_clustered.csv")
print("  4. cluster_summary.xlsx")
print("  5. clustering_metrics.csv")
print("\n" + "="*100)

: 